In [ ]:

import sys
from pathlib import Path
root = Path.cwd()
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from optuna import trial

if (root / "utils").exists():
    project_root = root
elif (root.parent / "utils").exists():
    project_root = root.parent

sys.path.insert(0, str(project_root))

from utils.data_preprocessing import (
    map_pistol_rounds,
    encode_categorical_columns
)
from utils.model_helpers import (
    train_lightgbm_model,
    model_traininng_pipeline

)

DROP_COLUMNS = ["demo_file", "tick", "user_name", "attacker_name"]
CATEGORICAL_COLS = ["weapon","attacker_team_name","user_team_name"]

In [ ]:
df = pd.read_csv("../../data/data/processed_demo.csv")

In [ ]:
display(df.head())

In [ ]:
df.drop(columns=DROP_COLUMNS, inplace=True)

In [ ]:
df = map_pistol_rounds(df)

In [ ]:
display(df)


In [ ]:
df = encode_categorical_columns(df, CATEGORICAL_COLS)[0].drop(columns=["round"])

In [ ]:
#display(df)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df.drop(columns="t_won_round"),df["t_won_round"],test_size=0.2)

In [ ]:
#display(X_train)

In [ ]:
#display(y_train)

In [ ]:
def create_parameter_space(trial):
    """Create custom parameter space for LightGBM hyperparameter optimization."""
    param_grid = {
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "max_bin": trial.suggest_int("max_bin", 200, 300),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
    }
    return param_grid


In [ ]:
#model, metrics = model_traininng_pipeline(
#    X_train, 
#    y_train, 
#    X_val, 
#    y_val, 
#    parameter_space=create_parameter_space,
#    n_trials=50
#)


In [ ]:
def metrics_to_df(metrics:dict) -> pd.DataFrame:
    df = pd.DataFrame.from_dict(metrics["classification_report"])
    df["auc"] = metrics["auc"]
    return df


In [ ]:
df_metrics = metrics_to_df(metrics)

In [ ]:
display(df_metrics)